In [1]:
#mapping as on kibana with custom analyzer as defined in the setting

# PUT product_details2
# {
#   "settings": {
#     "analysis": {
#       "analyzer": {
#         "custom": { 
#           "type": "custom",
#           "tokenizer": "standard",
#           "max_token_length": 5,   
#           "stopwords": "_english_",
#           "filter": [
#             "lowercase",
#             "asciifolding"
#           ]
#         }
#       }
#     }
#   },     
#   "mappings": {
#     "product2": {
#       "properties": {
#         "Transaction_date" :{"type": "text"},
#         "ProductID": {"type":"text"},
#         "Price": {"type":"text"},
#         "Payment_type":{"type":"text"},        
#         "Name": {"type":"text"},
#         "City": {"type":"text"},
#         "State": {"type":"text"},
#         "Country": {"type":"text","boost":5},
#         "Account_Created": {"type":"text"},
#         "Last_Login":{"type": "text"},
#         "Latitude":{"type": "text"},
#         "Longitude":{"type": "text"}
#       }
#     }  
#   }
# }



In [2]:
from elasticsearch import helpers, Elasticsearch
import csv

es = Elasticsearch()


In [3]:
with open('data_2.csv', 'rU') as f:
  reader = csv.DictReader(f)
  helpers.bulk(es, reader, index='product_details2', doc_type='product2')

In [16]:
#boolean-query with minimum-should match and boost-value 
booleanquery={
  "query": {
    "bool" : {
      "must" : {
        "match" : { "Country" : "Canada" }
      },
      "filter": {
        "match" : { "Product" : "Product1" }
      },
      "must_not" : {
        "match" : { "State" : "Ontario" }      
      },
      "should" : [
        { "match" : { "Country" : "Canada" } },
        { "match" : { "State" : "England" } }
      ],
      "minimum_should_match" : 1,
      "boost" : 2.0
    }
  }
}

res = es.search(index="product_details2",body=booleanquery)
for hit in res['hits']['hits']:
	print hit['_source']['Country'],hit['_source']['State'],hit['_source']['Product']
    

Canada British Columbia Product1
Canada British Columbia Product1
Canada British Columbia Product1
Canada Alberta Product1
Canada Alberta Product1
Canada British Columbia Product1
Canada Quebec Product1
Canada British Columbia Product1
Canada Alberta Product1
Canada British Columbia Product1
